In [34]:
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')

df = pd.read_excel('data/stanfordMOOCForumPostsSet.xlsx')
df.head(3)

,Text,Opinion(1/0),Question(1/0),Answer(1/0),Sentiment(1-7),Confusion(1-7),Urgency(1-7),CourseType,forum_post_id,course_display_name,forum_uid,created_at,post_type,anonymous,anonymous_to_peers,up_count,comment_thread_id,reads
0,Interesting! How often we say those things to ...,1,0,0,6.5,2.0,1.5,Education,5225177f2c501f0a00000015,Education/EDUC115N/How_to_Learn_Math,30CADB93E6DE4711193D7BD05F2AE95C,2013-09-02 22:55:59,Comment,0.0,0.0,0.0,5221a8262cfae31200000001,41.0
1,"What is \Algebra as a Math Game\"" or are you j...",0,1,0,4.0,5.0,3.5,Education,5207d0e9935dfc0e0000005e,Education/EDUC115N/How_to_Learn_Math,37D8FAEE7D0B94B6CFC57D98FD3D0BA5,2013-08-11 17:59:05,Comment,0.0,0.0,0.0,520663839df35b0a00000043,55.0
2,I like the idea of my kids principal who says ...,1,0,0,5.5,3.0,2.5,Education,52052c82d01fec0a00000071,Education/EDUC115N/How_to_Learn_Math,CC11480215042B3EB6E5905EAB13B733,2013-08-09 17:53:06,Comment,0.0,0.0,0.0,51e59415e339d716000001a6,25.0


In [35]:
from transformers import pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")

Device set to use cuda:0


In [36]:
df["Text"][2]

'I like the idea of my kids principal who says \\Smart doesn\'t mean easy, smart means working hard\\" and incorporating the idea of making mistakes into working hard."'

In [37]:
# traduzir um documento
translator(df["Text"][2], max_length=2048)[0]["translation_text"]

'Eu gosto da ideia do diretor dos meus filhos que diz “Smart não significa fácil, inteligente significa trabalhar duro” e incorpora a ideia de cometer erros para trabalhar duro.'

In [38]:
# teste de 10 traduções
df["Text"][0:10].apply(lambda x: translator(x, max_length=2048)[0]["translation_text"])

0    Interessante! Quantas vezes dizemos essas cois...
1    O que é "Álgebra como um Jogo de Matemática" o...
2    Eu gosto da ideia do diretor dos meus filhos q...
3    A partir de suas respostas, parece que os alun...
4    Os meninos adoraram a matemática, porque há li...
5    A ideia generalizada de que ser bom em matemát...
6    O módulo de revisão por pares ainda não está t...
7    Gostaria de começar o ano de folga compartilha...
8    Pedir aos alunos que ilustrem como os erros po...
9    Eu acho que isso cai no professor. Alguém me d...
Name: Text, dtype: object

In [39]:
# melhor uso da GPU
translated_posts = translator(
    df["Text"][0:10]
    .apply(lambda x: x if type(x) in (str, list) else "no text")
    .to_list(),
    max_length=2048,
)
translated_posts = [post["translation_text"] for post in translated_posts]
translated_posts

['Interessante! Quantas vezes dizemos essas coisas aos outros sem realmente entender o que estamos dizendo. Deve ter sido uma experiência poderosa! Excelente!',
 'O que é "Álgebra como um Jogo de Matemática" ou você está apenas dizendo que cria jogos que incorporam álgebra?',
 'Eu gosto da ideia do diretor dos meus filhos que diz “Smart não significa fácil, inteligente significa trabalhar duro” e incorpora a ideia de cometer erros para trabalhar duro.',
 'A partir de suas respostas, parece que os alunos realmente gostaram do poder que sentiam ser "livres" para resolver a matemática da maneira que quisessem. Eles usaram linguagem acadêmica como números em decomposição e números amigáveis e foram capazes de explicar o significado. "',
 'Os meninos adoraram a matemática, porque há liberdade, você pode fazer qualquer coisa." Que ótima maneira de ver matemática e números! Não acho que os alunos percebam que a matemática pode ser ensinada de maneira diferente. Eles só pensaram que é isso que

In [40]:
# traduzir dataset completo
translated_posts = translator(
    df["Text"]
    .apply(lambda x: x if type(x) in (str, list) else "no text")
    .to_list(),
    max_length=2048,
)
translated_posts = [post["translation_text"] for post in translated_posts]

# salvar
df.insert(1, column="Text_PT", value=translated_posts)
df.to_excel('data/stanfordMOOCForumPostsSet_PT.xlsx', index=False)
df.head(3)

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
